# SAE Dashboard Improvement

In [6]:
import sys 
sys.path.append("..")

from importlib import reload

import joseph

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)

LAYER_IDX, HEAD_IDX = (10, 7)


model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    # refactor_factored_attn_matrices=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)



# path = "checkpoints/ikig1wjm/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_32768.pkl"
# path="../artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096:v15/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096.pkl"#
# path="../artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096:v16/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096.pkl"
# path="../artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_24576:v56/final_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_24576.pkl"
# hacky solution to saved with cuda load on mps:
# path = "artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096:v13/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_4096.pkl"
# sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)
path = "checkpoints/399ihu5z/final_sparse_autoencoder_tiny-stories-2L-33M_blocks.1.attn.hook_q_4096.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)
from sae_training.sparse_autoencoder import SparseAutoencoder
from sae_training.config import LanguageModelSAERunnerConfig

# path = "checkpoints/peu1onjp/132669440_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_8192.pkl"
# path = "checkpoints/g2zrx9ho/final_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_8192.pkl"
# path = "artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_65536:v28/1076002816_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_65536.pkl" 


# with open(path, 'rb') as file:
#     state_dict = CPU_Unpickler(file).load()

# cfg = state_dict["cfg"].__dict__
# print(cfg)
# cfg["total_training_tokens"] = 5_000_000
# cfg["device"] = "mps"
# cfg["hook_point_layer"] = 10
# del cfg["d_sae"]
# del cfg["tokens_per_buffer"]
# cfg = LanguageModelSAERunnerConfig(**cfg)
# sparse_autoencoder = SparseAutoencoder(cfg)
# sparse_autoencoder.load_state_dict(state_dict["state_dict"])
# del state_dict
# del cfg

# new_path = "artifacts/sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_65536:v28/1076002816_sparse_autoencoder_gpt2-small_blocks.10.attn.hook_q_65536_mps.pt"
# sparse_autoencoder.save_model(new_path)



Loaded pretrained model gpt2-small into HookedTransformer


In [7]:

from sae_training.utils import LMSparseAutoencoderSessionloader
model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)



Loaded pretrained model tiny-stories-2L-33M into HookedTransformer
Moving model to device:  mps
Dataset is not tokenized! Updating config.


In [8]:
sae_in = activation_store.next_batch()
sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(sae_in)

In [9]:
feature_acts.shape

torch.Size([8192, 4096])

In [10]:
def replacement_hook(mlp_post, hook, encoder):
    mlp_post_reconstr = encoder(mlp_post)[0]
    return mlp_post_reconstr
hook_point = activation_store.cfg.hook_point


with torch.no_grad():
    batch_tokens = activation_store.get_batch_tokens()
    _, cache = model.run_with_cache(batch_tokens, prepend_bos=False)
    original_act = cache[sparse_autoencoder.cfg.hook_point][:,:,sparse_autoencoder.cfg.hook_point_head_index]
    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
        original_act
    )
    # del cache
    
    # all round metrics
    
    original_act = cache[sparse_autoencoder.cfg.hook_point][:,:,sparse_autoencoder.cfg.hook_point_head_index]
    per_token_l2_loss = (sae_out - original_act).pow(2).sum(dim=-1).squeeze()
    total_variance = original_act.pow(2).sum(-1)
    explained_variance = per_token_l2_loss/total_variance

    print("explained variance mean:", explained_variance.mean().item())
    print("explained_variance std:", explained_variance.std().item())
    
    
    per_token_l2_loss = (sae_out - original_act).pow(2).sum(dim=-1).squeeze()
    px.histogram(per_token_l2_loss.flatten().cpu().numpy()).show()
    
    l0 = (feature_acts > 0).float().sum(-1).detach()
    print("average l0", l0.mean().item())
    print("l0 std", l0.std().item())
    # px.histogram(l0.flatten().cpu().numpy()).show()
    
    l2_norm = original_act.pow(2).sum(-1).sqrt()
    print("l2 norm mean", l2_norm.mean().item())
    print("l2 norm std", l2_norm.std().item())
    px.histogram(l2_norm.flatten().cpu().numpy()).show()
    px.line(l2_norm.mean(0).cpu().numpy(), title="l2 norm by position").show()
    
    
    # by position
    px.line(l0.mean(0).cpu().numpy(), title="L0 by Position").show()
    px.line(per_token_l2_loss.mean(0).cpu().numpy(), title = "MSE Loss by Position").show()
    px.scatter(
        x = per_token_l2_loss.flatten().cpu().numpy(),
        y = l0.flatten().cpu().numpy(),
        color = np.arange(per_token_l2_loss.shape[1]).repeat(per_token_l2_loss.shape[0]),
        opacity=0.5,
        labels = {"color": "position", "x": "MSE Loss", "y": "L0"},
        title = "L0 vs MSE Loss",
        marginal_x="histogram",
        marginal_y="histogram",
    ).show()
    
    px.scatter(
        x = l2_norm.flatten().cpu().numpy(),
        y =  per_token_l2_loss.flatten().cpu().numpy(),
        color = np.arange(per_token_l2_loss.shape[1]).repeat(per_token_l2_loss.shape[0]),
        opacity=0.5,
        labels={"color": "position", "x": "L2 Norm", "y": "MSE Loss"},
        title = "MSE Loss vs L2 Norm",
        marginal_x="histogram",
        marginal_y="histogram",
    ).show()
    


explained variance mean: 0.21688123047351837
explained_variance std: 0.09474866837263107


average l0 21.786376953125
l0 std 12.650598526000977
l2 norm mean 3.0572257041931152
l2 norm std 0.3278023898601532


In [ ]:
import numpy as np

# Assuming l2_norm is a 2D tensor with shape [num_samples, num_positions]
mean_l2_norm = l2_norm.mean(0).cpu().numpy()
std_l2_norm = l2_norm.std(0).cpu().numpy()
num_samples = l2_norm.shape[0]

# Calculate the standard error of the mean
sem_l2_norm = std_l2_norm / np.sqrt(num_samples)

# 95% confidence intervals
ci = 1.96 * sem_l2_norm

# Modify the line plot command to include error bars
px.line(
    y=mean_l2_norm, 
    error_y=ci, 
    title="l2 norm by position"
).show()

In [ ]:
def replacement_hook(mlp_post, hook, encoder):
    mlp_post_reconstr = encoder(mlp_post)[0]
    return mlp_post_reconstr
hook_point = activation_store.cfg.hook_point


def mean_ablate_hook(mlp_post, hook):
    mlp_post[:] = mlp_post.mean([0, 1])
    return mlp_post


def zero_ablate_hook(mlp_post, hook):
    mlp_post[:] = 0.0
    return mlp_post


def kl_divergence_attention(y_true, y_pred):

    # Compute log probabilities for KL divergence
    log_y_true = torch.log2(y_true + 1e-10)
    log_y_pred = torch.log2(y_pred + 1e-10)

    return y_true * (log_y_true - log_y_pred)

with torch.no_grad():
    batch_tokens = activation_store.get_batch_tokens()
    _, cache = model.run_with_cache(batch_tokens, prepend_bos=False)
    original_act = cache[sparse_autoencoder.cfg.hook_point][:,:,sparse_autoencoder.cfg.hook_point_head_index]
    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
        original_act
    )

    # get attn when using reconstructed activations
    with model.hooks(fwd_hooks=[(hook_point, partial(replacement_hook, encoder=sparse_autoencoder))]):
        _, new_cache = model.run_with_cache(batch_tokens)
        
    # get attn when using reconstructed activations
    with model.hooks(fwd_hooks=[(hook_point, partial(replacement_hook, encoder=sparse_autoencoder))]):
        _, zero_ablation_cache = model.run_with_cache(batch_tokens)


    # get the attention scores

    patterns_original = cache[utils.get_act_name("pattern", LAYER_IDX)][:,HEAD_IDX].detach().cpu()
    patterns_reconstructed = new_cache[utils.get_act_name("pattern", LAYER_IDX)][:,HEAD_IDX].detach().cpu()
    patterns_ablation = zero_ablation_cache[utils.get_act_name("pattern", LAYER_IDX)][:,HEAD_IDX].detach().cpu()
    
    # show patterns before/after
    px.imshow(patterns_original.numpy(), animation_frame = 0, title="original attn scores", width=800, height=800,
          color_continuous_midpoint=0, color_continuous_scale="RdBu").show()
    px.imshow(patterns_reconstructed.numpy(), animation_frame = 0, title="reconstructed attn scores", width=800, height=800,
            color_continuous_midpoint=0, color_continuous_scale="RdBu").show()
    
    # get the average non-bos_attn_score
    score_original = cache[utils.get_act_name("attn_scores", LAYER_IDX)][:,HEAD_IDX].detach().cpu()
    score_reconstructed = new_cache[utils.get_act_name("attn_scores", LAYER_IDX)][:,HEAD_IDX].detach().cpu()
    
    print(patterns_original[:,:,1:,].mean().item())
    print(patterns_reconstructed[:,:,1:].mean().item())
    
    # ratio
    kl_result = kl_divergence_attention(patterns_original, patterns_reconstructed)
    kl_result = kl_result.sum(dim=-1).numpy()
    print(kl_result.mean().item())
    # px.imshow(kl_result, title="KL Divergence", width=800, height=800,
    #       color_continuous_midpoint=0, color_continuous_scale="RdBu").show()
    px.histogram(kl_result.flatten()).show()
    px.line(kl_result.mean(0), title="KL Divergence by Position").show()
    
    kl_result = kl_divergence_attention(patterns_original, patterns_ablation)
    kl_result = kl_result.sum(dim=-1).numpy()
    print(kl_result.mean().item())
    # px.imshow(kl_result, title="KL Divergence", width=800, height=800,
    #       color_continuous_midpoint=0, color_continuous_scale="RdBu").show()
    px.histogram(kl_result.flatten()).show()
    px.line(kl_result.mean(0), title="KL Divergence by Position").show()
    

In [ ]:
score_original.shape

In [ ]:
HEAD_IDX